## Executive Summary

### GOAL:
Discover the distribution of landscape suitability for Piping Plovers across the Great Lakes region of the USA? 

### Metrics:
What are the landscape metrics associated with Piping Plover observation locations in this region?
AUCROC will be used as a scoring measure due to the resampling requirement. 

### Potential Audience: 
To help environmental managers determine habitat characteristics and locations for Piping Plovers

### Data Sources:

<ul>
<li>The eBird Reference Dataset Version 2016:
    <ul>
    <li>checklist (bird observations of all species)
    <li>extended covarites (associated physical landscape features linked by sample_id)
    </ul>
    
<li>Great Lakes Coastal Wetlands Consortium:
    <ul>
    <li>wetlands
    <li>wetlands information
    <li>shoreline classes
    <li>shore polygon (lakes)
    <li>islands
    <li>reefs
    <li>roads for Canada and USA
    </ul>           
    
<li>Features derived using ArcGIS "Near" tool / distance to nearest physical features:<ul> 
        <li>wetlands, 
        <il>shorelines,
        <il>islands,
        <il>reefs,
        <il>roads
        </ul> 
        
<li>Scientific Journals for domain knowledge

</ul> 
### Findings:
The project discovered key physical landscape features that are associated with Piping Plover sightings. Namely distance from shorelines are ideally closer than 150m, and there is a preference for sandy shorelines. 

### Assumptions and Risks:
The project is still a work in progress, and further refinement of the model is required. 


<hr>

## import required packages

In [1]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
sns.set_style('whitegrid')
import glob
import folium
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print('imports ok')

imports ok


## Load the data
<ul>
<li>use glob to read multiple files for all bird observations 2001 to 2016 for the USA
<li>read csv files with chunksize used to limit the amount of RAM used up.
</ul>

In [89]:
# use glob to handle multiple files

filenames = glob.glob('ERD2016SS/*/checklists.csv')
#            'ERD2016SS/2003/checklists.csv'

dfchunk = pd.DataFrame()
chunksize = 10**5
for file in filenames:
    i = 0
    for f in pd.read_csv(file, chunksize=chunksize, usecols=['SAMPLING_EVENT_ID','LOC_ID','LATITUDE','LONGITUDE','YEAR','MONTH','DAY','TIME','COUNTRY','STATE_PROVINCE','COUNTY','COUNT_TYPE','EFFORT_HRS','EFFORT_DISTANCE_KM','EFFORT_AREA_HA','OBSERVER_ID','NUMBER_OBSERVERS','GROUP_ID','PRIMARY_CHECKLIST_FLAG','Charadrius_melodus']):
        dfchunk = dfchunk.append(f[f.Charadrius_melodus != 0])
        i+=1
    print(file + " has been imported.", " Chunks: ", i, " Shape: ", f.shape)
       
dfalpha = dfchunk[dfchunk.Charadrius_melodus != '0']
dfnum = dfalpha[dfalpha.Charadrius_melodus != 'X']
dfnum.Charadrius_melodus = pd.to_numeric(dfnum.Charadrius_melodus) 
print('Shape of Alphabetic Dataframe: ', dfalpha.shape)
print('Shape of Numeric DataFrame:    ', dfnum.shape)
print(dfalpha['Charadrius_melodus'].unique())
print(dfnum['Charadrius_melodus'].unique())


C:\Users\Wayne\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (18,849) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Wayne\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (13,18,849) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


ERD2016SS\2002\checklists.csv has been imported.  Chunks:  5  Shape:  (12994, 20)
ERD2016SS\2003\checklists.csv has been imported.  Chunks:  1  Shape:  (60405, 20)
ERD2016SS\2004\checklists.csv has been imported.  Chunks:  1  Shape:  (67771, 20)
ERD2016SS\2005\checklists.csv has been imported.  Chunks:  1  Shape:  (81581, 20)
ERD2016SS\2006\checklists.csv has been imported.  Chunks:  2  Shape:  (31601, 20)
ERD2016SS\2007\checklists.csv has been imported.  Chunks:  2  Shape:  (91411, 20)
ERD2016SS\2008\checklists.csv has been imported.  Chunks:  3  Shape:  (61967, 20)
ERD2016SS\2009\checklists.csv has been imported.  Chunks:  5  Shape:  (34597, 20)
ERD2016SS\2010\checklists.csv has been imported.  Chunks:  7  Shape:  (61010, 20)


C:\Users\Wayne\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (849) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


ERD2016SS\2011\checklists.csv has been imported.  Chunks:  10  Shape:  (5543, 20)
ERD2016SS\2012\checklists.csv has been imported.  Chunks:  14  Shape:  (51923, 20)
ERD2016SS\2013\checklists.csv has been imported.  Chunks:  18  Shape:  (63876, 20)
ERD2016SS\2014\checklists.csv has been imported.  Chunks:  23  Shape:  (86406, 20)
ERD2016SS\2015\checklists.csv has been imported.  Chunks:  29  Shape:  (503, 20)
ERD2016SS\2016\checklists.csv has been imported.  Chunks:  34  Shape:  (17024, 20)
Shape of Alphabetic Dataframe:  (62743, 20)
Shape of Numeric DataFrame:     (59534, 20)
[2 6 1 4 20 18 10 'X' '4' 16 8 5 15 3 12 40 '1' '10' '2' 9 '8' 35 11 58 14
 13 '6' 21 7 '5' '3' 25 29 36 30 22 '33' '9' '50' '20' 17 '15' 46 '14'
 '11' '17' '24' 172 '7' '16' '126' '40' '59' 39 '32' 24 26 45 38 32 28 42
 74 34 19 23 47 210 51 48 31 62 66 60 165 63 43 79 37 53 49 41 '30' 27 33
 44 52 '12' 64 50 '58' 150 196 55 '25' '53' '86' '76' '62' '35' '55' '43'
 '41' '52' 70 76 401 59 200 75 54 '200' 101 100 '

C:\Users\Wayne\Anaconda3\lib\site-packages\pandas\core\generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


### two versions saved to working files, one for presence only and another for numerical counts in case it is neeeded. 

In [90]:
dfnum.to_csv('piping_plover_num.csv')

In [91]:
dfalpha.to_csv('piping_plover_alpha.csv')

### remove duplicated observations done by birders working in groups:

In [98]:


#dfalpha = dfalpha[dfalpha.PRIMARY_CHECKLIST_FLAG != '?']
print(dfalpha.PRIMARY_CHECKLIST_FLAG.unique())
#dfnum = dfnum[dfnum.PRIMARY_CHECKLIST_FLAG != '?']
print(dfnum.PRIMARY_CHECKLIST_FLAG.unique())
print('Shape of Alphabetic Dataframe: ', dfalpha.shape)
print('Shape of Numeric DataFrame:    ', dfnum.shape)
dfalpha.PRIMARY_CHECKLIST_FLAG = pd.to_numeric(dfalpha.PRIMARY_CHECKLIST_FLAG)
dfnum.PRIMARY_CHECKLIST_FLAG = pd.to_numeric(dfnum.PRIMARY_CHECKLIST_FLAG)

[1]
[1]
Shape of Alphabetic Dataframe:  (49350, 20)
Shape of Numeric DataFrame:     (46489, 20)


In [99]:
dfnum.to_csv('piping_plover_num.csv')
dfalpha.to_csv('piping_plover_alpha.csv')

### verify that sampling IDs are unique values

In [113]:
# check that sampling event IDs are all unique
temp = pd.DataFrame()
temp['is_duplicated'] = dfalpha.duplicated('SAMPLING_EVENT_ID')
print('duplicate keys: ', temp['is_duplicated'].sum())



duplicate keys:  0


### check for duplicate sample id records in the covariates files. 

In [134]:

#filenames =  glob.glob('ERD2016SS/reference/2016/extended-covariates-sample.csv') # test file only
filenames = glob.glob('ERD2016SS/*/extended-covariates.csv')

temp = pd.DataFrame()
dfcov = pd.DataFrame()
for file in filenames:
    dfcov = pd.read_csv(file)
    temp['is_duplicated'] = dfcov.duplicated('SAMPLING_EVENT_ID')
    print(file + " has been read.", "Shape: ", dfcov.shape)
    print('duplicate keys: ', temp['is_duplicated'].sum())

ERD2016SS\2002\extended-covariates.csv has been read. Shape:  (412994, 92)
duplicate keys:  0
ERD2016SS\2003\extended-covariates.csv has been read. Shape:  (60405, 92)
duplicate keys:  0
ERD2016SS\2004\extended-covariates.csv has been read. Shape:  (67771, 92)
duplicate keys:  0
ERD2016SS\2005\extended-covariates.csv has been read. Shape:  (81581, 92)
duplicate keys:  0
ERD2016SS\2006\extended-covariates.csv has been read. Shape:  (131601, 92)
duplicate keys:  0
ERD2016SS\2007\extended-covariates.csv has been read. Shape:  (191411, 92)
duplicate keys:  0
ERD2016SS\2008\extended-covariates.csv has been read. Shape:  (261967, 92)
duplicate keys:  0
ERD2016SS\2009\extended-covariates.csv has been read. Shape:  (434597, 92)
duplicate keys:  0
ERD2016SS\2010\extended-covariates.csv has been read. Shape:  (661010, 92)
duplicate keys:  0
ERD2016SS\2011\extended-covariates.csv has been read. Shape:  (905543, 92)
duplicate keys:  0
ERD2016SS\2012\extended-covariates.csv has been read. Shape:  (

### read covariates files using glob and chunksize, saving them to one file. 

In [2]:
filenames = glob.glob('../ERD2016SS/*/extended-covariates.csv')
output = "../Datasets/xcovariates.csv"
print(output)
chunk = pd.DataFrame()
chunksize = 10**4
with open(output, 'a') as outfile:
        
    for file in filenames:
        i = 0
        for chunk in pd.read_csv(file, chunksize=chunksize):
            chunk.to_csv(outfile)
            i+=1
        print(file + " has been imported.", " Chunks: ", i)
    
print("done")

../Datasets/xcovariates.csv
../ERD2016SS\2002\extended-covariates.csv has been imported.  Chunks:  42
../ERD2016SS\2003\extended-covariates.csv has been imported.  Chunks:  7
../ERD2016SS\2004\extended-covariates.csv has been imported.  Chunks:  7
../ERD2016SS\2005\extended-covariates.csv has been imported.  Chunks:  9
../ERD2016SS\2006\extended-covariates.csv has been imported.  Chunks:  14
../ERD2016SS\2007\extended-covariates.csv has been imported.  Chunks:  20
../ERD2016SS\2008\extended-covariates.csv has been imported.  Chunks:  27
../ERD2016SS\2009\extended-covariates.csv has been imported.  Chunks:  44
../ERD2016SS\2010\extended-covariates.csv has been imported.  Chunks:  67
../ERD2016SS\2011\extended-covariates.csv has been imported.  Chunks:  91
../ERD2016SS\2012\extended-covariates.csv has been imported.  Chunks:  136
../ERD2016SS\2013\extended-covariates.csv has been imported.  Chunks:  177
../ERD2016SS\2014\extended-covariates.csv has been imported.  Chunks:  229
../ERD2016

## DATA DICTIONARIES

### Created a heatmap showing Piping Plover observations

In [10]:
import csv
from folium import plugins
def is_number(n):
    try:
        
        float(n)   # Type-casting the string to `float`.
                   # If string is not a valid `float`, 
                   # it'll raise `ValueError` exception
                
    except ValueError:
        return False
    return True
data=[]
heatmap = folium.Map()
with open('../Datasets/selected_observations.csv','r') as f: 
    reader = csv.reader(f)
    for row in reader:
        if is_number(row[4]):
            x = float(row[4])
            if is_number(row[5]):
                y = float(row[5])
                data.append([x, y])
h = plugins.HeatMap(data)
heatmap.add_child(h)
f.close()
heatmap.save("heatmap.html")
heatmap

### planned to use SQLite, however decided that it wasn't required at a later stage

In [5]:
import sqlite3
conn=sqlite3.connect('pipingplovers.db')
print ("Database created and opened succesfully")


Database created and opened succesfully


### Read files from "iplover" source, however decided to focus on Great Lakes area. "iplover" covered a different area.

In [36]:
csvfile = "Datasets\iplover_fieldpts_2015_csv\iplover_fieldpts_2015.csv"
df = pd.read_csv(csvfile)
df = df.drop(['collectiongroup','deviceinfo'], axis=1)
table_name = "iplover_fieldpts"
df.to_sql(table_name, conn, if_exists='append', index=False)
df.head()

,id,clientversion,datetime,latitude,longitude,accuracy,site,setting,vegtype,vegdens,substrate,imagefile,setting_2,vegtype_2,vegdens_2,substrate_2
0,6a99d68c-dcc0-4d82-ad54-afd6c233ad31,2.0-SNAPSHOT,8/15/15 9:58,41.347731,-70.645439,5.0,Cra_R17,beach,none,none,water,6a99d68c-dcc0-4d82-ad54-afd6c233ad31.jpg,Beach,None,None,Water
1,b7f34b94-98a5-4983-be10-8c65274a012d,2.0-SNAPSHOT,8/15/15 9:55,41.347668,-70.646774,5.0,Cra_R2S,beach,none,none,water,b7f34b94-98a5-4983-be10-8c65274a012d.jpg,Beach,None,None,Water
2,ef579f57-a105-466d-8f5b-b0ee87508d0a,2.0-SNAPSHOT,8/14/15 16:29,41.422426,-70.705820,5.0,SSG_R2,beach,none,none,water,ef579f57-a105-466d-8f5b-b0ee87508d0a.jpg,Beach,None,None,Water
3,4e1f32a4-cd86-4dda-97d0-7887ae1d02e0,2.0-SNAPSHOT,8/14/15 16:17,41.426118,-70.703443,5.0,SSG_R6S,dune,forest,dense,unknown,4e1f32a4-cd86-4dda-97d0-7887ae1d02e0.jpg,Dune,Forest,Dense,*
4,1193fb59-2bf3-4576-9947-8e25b5a96ffd,2.0-SNAPSHOT,8/14/15 14:47,41.831606,-69.947982,5.0,Random_CoGu_8S,swale,none,none,sandy,1193fb59-2bf3-4576-9947-8e25b5a96ffd.jpg,RidgeSwale,None,None,Sand


In [6]:
csvfile = "Datasets\iplover_fieldpts_2014_csv\iplover_fieldpts_2014.csv"
df = pd.read_csv(csvfile)
table_name = "iplover_fieldpts"
df.to_sql(table_name, conn, if_exists='append', index=False)
df.head()

,id,clientversion,datetime,latitude,longitude,accuracy,site,setting,vegtype,vegdens,substrate,imagefile,setting_2,vegtype_2,vegdens_2,substrate_2
0,218,1.2-SNAPSHOT,2014-06-10 11:51,37.165681,-75.847508,5.0,smith island,barrierinterior,shellbed,sparse,sandy,MISSING,BarrierInterior,None,Sparse,ShellGravelCobble
1,219,1.2-SNAPSHOT,2014-06-10 11:24,37.168905,-75.842584,5.0,smith island,washover,shellbed,sparse,sandy,MISSING,Washover,None,Sparse,ShellGravelCobble
2,478,1.2-SNAPSHOT,2014-06-25 13:47,40.552922,-73.940334,5.0,nest 151,lowduneforedune,herbaceous,sparse,sandy,hQFds6KpCC7anyo2.jpg,Dune,Herbaceous,Sparse,Sand
3,221,1.2-SNAPSHOT,2014-06-10 10:55,37.170426,-75.840059,5.0,smith island,washover,shellbed,sparse,sandy,ShpLBvqm6vaiJ1Dd.jpg,Washover,None,Sparse,ShellGravelCobble
4,222,1.2-SNAPSHOT,2014-06-10 10:46,37.170897,-75.839414,5.0,smith island,washover,shellbed,none,sandy,A0GoE08ii3OdVckc.jpg,Washover,None,None,ShellGravelCobble


### write files to SQLite (not used later on)

In [38]:
filenames = glob.glob('ERD2016SS/*/extended-covariates.csv')
table_name = "covariates" 
dfchunk = pd.DataFrame()
chunksize = 10**5
for file in filenames:
    i = 0
    for chunk in pd.read_csv(file, chunksize=chunksize):
        chunk.to_sql(table_name, conn, if_exists='append', index=False)
        i+=1
    print(file + " has been imported.", " Chunks: ", i, " Shape: ", chunk.shape)

ERD2016SS\2002\extended-covariates.csv has been imported.  Chunks:  5  Shape:  (12994, 92)
ERD2016SS\2003\extended-covariates.csv has been imported.  Chunks:  1  Shape:  (60405, 92)
ERD2016SS\2004\extended-covariates.csv has been imported.  Chunks:  1  Shape:  (67771, 92)
ERD2016SS\2005\extended-covariates.csv has been imported.  Chunks:  1  Shape:  (81581, 92)
ERD2016SS\2006\extended-covariates.csv has been imported.  Chunks:  2  Shape:  (31601, 92)
ERD2016SS\2007\extended-covariates.csv has been imported.  Chunks:  2  Shape:  (91411, 92)
ERD2016SS\2008\extended-covariates.csv has been imported.  Chunks:  3  Shape:  (61967, 92)
ERD2016SS\2009\extended-covariates.csv has been imported.  Chunks:  5  Shape:  (34597, 92)
ERD2016SS\2010\extended-covariates.csv has been imported.  Chunks:  7  Shape:  (61010, 92)
ERD2016SS\2011\extended-covariates.csv has been imported.  Chunks:  10  Shape:  (5543, 92)
ERD2016SS\2012\extended-covariates.csv has been imported.  Chunks:  14  Shape:  (51923, 92